In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = r'C:\Users\yakam\Downloads\Fertilizer Prediction.csv'
df = pd.read_csv(file_path)

# Check the available columns in the dataset
print("Columns in the dataset:", df.columns)

# Correct column names if necessary
df.rename(columns={
    'Humidity ': 'Humidity',
    'Soil Type': 'Soil_Type',
    'Crop Type': 'Crop_Type',
    'Fertilizer Name': 'Fertilizer'
}, inplace=True)

# Print the first few rows to verify the data
print("First few rows of the dataset:")
print(df.head())

# Encode categorical variables
label_encoder_soil = LabelEncoder()
df['Soil_Type'] = label_encoder_soil.fit_transform(df['Soil_Type'])

label_encoder_crop = LabelEncoder()
df['Crop_Type'] = label_encoder_crop.fit_transform(df['Crop_Type'])

label_encoder_fertilizer = LabelEncoder()
df['Fertilizer'] = label_encoder_fertilizer.fit_transform(df['Fertilizer'])

# Store mappings for later use
crop_type_mapping = dict(zip(label_encoder_crop.classes_, label_encoder_crop.transform(label_encoder_crop.classes_)))
fertilizer_name_mapping = dict(zip(label_encoder_fertilizer.transform(label_encoder_fertilizer.classes_), label_encoder_fertilizer.classes_))

# Define features and target variable
X = df[['Soil_Type', 'Crop_Type', 'Humidity', 'Moisture']]  # Adjust based on available columns
y = df['Fertilizer']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Function to predict fertilizer based on crop name and soil type
def predict_fertilizer(crop_name, Soil_Type, humidity, moisture):
    # Encode the input values
    crop_encoded = label_encoder_crop.transform([crop_name])[0]
    soil_encoded = label_encoder_soil.transform([Soil_Type])[0]
    
    # Create a DataFrame for the input
    input_data = pd.DataFrame([[soil_encoded, crop_encoded, humidity, moisture]], 
                              columns=['Soil_Type', 'Crop_Type', 'Humidity', 'Moisture'])
    
    # Make a prediction
    fertilizer_encoded = model.predict(input_data)[0]
    
    # Decode the predicted fertilizer name
    fertilizer_name = label_encoder_fertilizer.inverse_transform([fertilizer_encoded])[0]
    return fertilizer_name

# Example usage
crop_name = 'Wheat'  # Replace with actual crop name
Soil_Type = 'Red'   # Replace with actual soil type
humidity = 50        # Replace with actual humidity
moisture = 30        # Replace with actual moisture

recommended_fertilizer = predict_fertilizer(crop_name, Soil_Type,humidity, moisture)
print(f"Recommended Fertilizer for {crop_name}: {recommended_fertilizer}")


Columns in the dataset: Index(['Temparature', 'Humidity ', 'Moisture', 'Soil Type', 'Crop Type',
       'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name'],
      dtype='object')
First few rows of the dataset:
   Temparature  Humidity  Moisture Soil_Type  Crop_Type  Nitrogen  Potassium  \
0           26        52        38     Sandy      Maize        37          0   
1           29        52        45     Loamy  Sugarcane        12          0   
2           34        65        62     Black     Cotton         7          9   
3           32        62        34       Red    Tobacco        22          0   
4           28        54        46    Clayey      Paddy        35          0   

   Phosphorous Fertilizer  
0            0       Urea  
1           36        DAP  
2           30   14-35-14  
3           20      28-28  
4            0       Urea  
Recommended Fertilizer for Wheat: 28-28
